In [14]:
# --- IMPORTS ---
import cv2
from ultralytics import YOLO
import easyocr
import numpy as np
import re

In [15]:
# --- INITIALIZATION ---

# Load the YOLO model
model = YOLO('35epoch.pt')

# Initialize EasyOCR Reader
reader = easyocr.Reader(['en'], gpu=True) # Use gpu=False if there is no CUDA-enabled GPU

# Open the video file
video_path = 'test.MOV'
cap = cv2.VideoCapture(video_path)

In [16]:
# --- CONFIGURATION FOR LABELS ---
CLASS_CONFIG = {
    3: {"label": "Plate", "color": (0, 255, 0)},      # for LicensePlate (ID: 3)
    2: {"label": "Expires", "color": (255, 165, 0)}   # for ExpiredDate (ID: 2)
}

In [17]:
# --- VIDEO WRITER SETUP ---
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = 'final_output_video_validated.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

In [18]:
# --- VIDEO PROCESSING LOOP ---

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, stream=True)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])

            if class_id in CLASS_CONFIG:
                config = CLASS_CONFIG[class_id]
                label_prefix = config["label"]
                color = config["color"]
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cropped_region = frame[y1:y2, x1:x2]
                
                ocr_text = "..." # Default text

                try:
                    ocr_results = reader.readtext(cropped_region, detail=0, paragraph=False)
                    raw_text = "".join(ocr_results).upper().replace(" ", "")

                    # --- LOGIC FOR DIFFERENT LABELS ---
                    if class_id == 3: # LicensePlate
                        ocr_text = raw_text if raw_text else "..."

                    elif class_id == 2: # ExpiredDate
                        # <<< NEW: VALIDATE DATE FORMAT (MM.YY) >>>
                        if re.match(r"^\d{2}\.\d{2}$", raw_text):
                            ocr_text = raw_text # Format is correct
                        else:
                            ocr_text = "INVALID" # Format is incorrect
                        # <<< END NEW SECTION >>>

                except Exception as e:
                    ocr_text = "ERROR"

                display_text = f"{label_prefix}: {ocr_text}"

                # Draw the bounding box and the recognized text
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                (text_width, text_height), _ = cv2.getTextSize(display_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1 - 5), color, -1)
                cv2.putText(frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    # Write the frame and display it
    out.write(frame)
    # cv2.imshow('License Plate Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- CLEAN UP ---
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 1 Car, 1 LicensePlate, 94.8ms
Speed: 7.9ms preprocess, 94.8ms inference, 10.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Car, 1 LicensePlate, 47.3ms
Speed: 2.0ms preprocess, 47.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)



/opt/anaconda3/envs/h8_env/lib/python3.9/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


0: 384x640 1 Car, 1 LicensePlate, 39.8ms
Speed: 2.8ms preprocess, 39.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Car, 1 LicensePlate, 52.2ms
Speed: 26.9ms preprocess, 52.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Car, 1 LicensePlate, 38.6ms
Speed: 1.8ms preprocess, 38.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 1 LicensePlate, 40.1ms
Speed: 1.8ms preprocess, 40.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 1 LicensePlate, 39.7ms
Speed: 1.7ms preprocess, 39.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Car, 1 LicensePlate, 42.8ms
Speed: 2.7ms preprocess, 42.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 1 LicensePlate, 38.7ms
Speed: 1.7ms preprocess, 38.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 1 Li

In [19]:
print(f"Processed video saved to: {output_path}")

Processed video saved to: final_output_video_validated.mp4
